In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick


from pymatgen import Lattice, Structure, Molecule
from pymatgen.io.vasp import outputs as vaspout
from pymatgen.io.vasp import inputs as vaspin
from pymatgen.symmetry import analyzer
from random import shuffle

import pymatgen as pmg
from pymatgen.io.vasp import outputs as vasp
from pymatgen.core.surface import Slab, SlabGenerator
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
from pymatgen.transformations import standard_transformations
from scipy.interpolate import interp1d

In [2]:
a_Mg = 2.0996784416305903 * 2**0.5
c_Mg = 2.0996784416305903 * 3**0.5 * 2
c_Mg_on_Zn = 7.0101798638883475

a_Ca = 2.4038016181676580 *2**0.5
c_Ca = 2.4038016181676580 *3**0.5 * 2
c_Ca_on_Zn = 8.4973270615965415


a_Cd = 2.3614121609750778 *2**0.5
c_Cd = 2.3614121609750778 *3**0.5 * 2
c_Cd_on_Zn = 8.3518376319009864

a_Zn = 3.2562112433332855
c_Zn = 5.2436599575602845

e_Zn = 3.78 # https://www.vasp.at/mmars/day2.pdf
e_Mg = 2.94 # https://arxiv.org/pdf/1706.08159.pdf

e_clamp_Ca00 = 2.985
e_clamp_Ca00s = 2.986
e_clamp_Ca50 = 3.316

e_clamp_Cd00 = 3.773
e_clamp_Cd00s = 3.724
e_clamp_Cd50 = 5.153

e_clamp_Mg00 = 3.092
e_clamp_Mg00s = 3.157
e_clamp_Mg50 = 2.910

e_clamp_Zn00 = 3.567
e_clamp_Zn25 = 3.743

In [3]:
a_Zn / 2

1.6281056216666427

In [4]:
a_Zn / 2 * 3**0.5

2.8199616568151376

In [5]:
5.2436599575602845 /2

2.6218299787801422

In [6]:
nZn = 8
nMg = 8

InitialZnO = Structure.from_file('/home/nadamski/ZnO.vasp')

ZnMg = InitialZnO * [[1, 0, 0], [0, 1, 0], [0, 0, nZn+nMg]]

Mg = c_Cd_on_Zn / 3
Zn = c_Zn / 2

cations = Mg * nMg + Zn * nZn

strain = cations / ZnMg.lattice.c

ZnMg.apply_strain([0, 0, strain-1])

wz = 0 #1-2*(0.4996673007200840 - 0.1200226992799145)
wz2 = 0.5

A = [2/3, 1/3]
B = [1/3, 2/3]
C = [0, 0]

for at in np.arange(0, nZn, 2):
    at = int(at)
    ZnMg[at] = 'Zn', A + [at*Zn/cations]
    ZnMg[2*nZn+at] = 'O', B + [(at -wz)*Zn/cations]
for at in np.arange(1, nZn, 2):
    at = int(at)
    ZnMg[at] = 'Zn', B + [at*Zn/cations]
    ZnMg[2*nZn+at] = 'O', A + [(at -wz)*Zn/cations]

    
for at in np.arange(nZn+2, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Cd', A + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[2*nZn+at] = 'O', C + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]

for at in np.arange(nZn, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Cd', C + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[2*nZn+at] = 'O', B + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]

for at in np.arange(nZn+1, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Cd', B + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[2*nZn+at] = 'O', A + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]
    
ZnMg[-1] = 'O', A + [(-1 -wz2)*Mg/cations]
ZnMg[nMg+nZn] = 'O', C + [(-wz2)*Mg/cations]
ZnMg[nZn*2+nMg] = 'O', B + [(nZn -wz/2 - wz2/2)*Zn/cations]



ZnMg.to(fmt='poscar',filename='/home/nadamski/strain.vasp')

In [32]:
nZn = 6
nMg = 8

InitialZnO = Structure.from_file('/home/nadamski/ZnO.vasp')

ZnMg = InitialZnO * [[1, 0, 0], [0, 1, 0], [0, 0, nZn+nMg]]

Mg = c_Mg_on_Zn / 3
Zn = c_Zn / 2

cations = Mg * nMg + Zn * nZn

strain = cations / ZnMg.lattice.c

ZnMg.apply_strain([0, 0, strain-1])

wz = 0 #1-2*(0.4996673007200840 - 0.1200226992799145)
wz2 = 0.0

A = [2/3, 1/3]
B = [1/3, 2/3]
C = [0, 0]

for at in np.arange(0, nZn, 2):
    at = int(at)
    ZnMg[at] = 'Zn', A + [at*Zn/cations]
    ZnMg[nMg+nZn+at] = 'O', B + [(at -wz)*Zn/cations]
for at in np.arange(1, nZn, 2):
    at = int(at)
    ZnMg[at] = 'Zn', B + [at*Zn/cations]
    ZnMg[nMg+nZn+at] = 'O', A + [(at -wz)*Zn/cations]

    
for at in np.arange(nZn+2, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Mg', A + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[nMg+nZn+at] = 'O', C + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]

for at in np.arange(nZn, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Mg', C + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[nMg+nZn+at] = 'O', B + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]

for at in np.arange(nZn+1, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Mg', B + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[nMg+nZn+at] = 'O', A + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]
    
ZnMg[-1] = 'O', A + [(-1 -wz2)*Mg/cations]
ZnMg[nMg+nZn] = 'O', C + [(-wz2)*Mg/cations]
ZnMg[nZn*2+nMg] = 'O', B + [(nZn - wz2)*Zn/cations]



ZnMg.to(fmt='poscar',filename='/home/nadamski/strain.vasp')

In [29]:
nZn = 8
nMg = 8

InitialZnO = Structure.from_file('/home/nadamski/ZnO.vasp')

ZnMg = InitialZnO * [[1, 0, 0], [0, 1, 0], [0, 0, nZn+nMg]]

Mg = c_Mg_on_Zn / 3
Zn = c_Zn / 2

cations = Mg * nMg + Zn * nZn

strain = cations / ZnMg.lattice.c

ZnMg.apply_strain([0, 0, strain-1])

wz = 0 #1-2*(0.4996673007200840 - 0.1200226992799145)
wz2 = 0.5

A = [2/3, 1/3]
B = [1/3, 2/3]
C = [0, 0]

for at in np.arange(0, nZn, 2):
    at = int(at)
    ZnMg[at] = 'Zn', A + [at*Zn/cations]
    ZnMg[2*nZn+at] = 'O', B + [(at -wz)*Zn/cations]
for at in np.arange(1, nZn, 2):
    at = int(at)
    ZnMg[at] = 'Zn', B + [at*Zn/cations]
    ZnMg[2*nZn+at] = 'O', A + [(at -wz)*Zn/cations]

    
for at in np.arange(nZn+2, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Mg', A + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[2*nZn+at] = 'O', C + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]

for at in np.arange(nZn, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Mg', C + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[2*nZn+at] = 'O', B + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]

for at in np.arange(nZn+1, nZn+nMg, 3):
    at = int(at)
    ZnMg[at] = 'Mg', B + [(at-nZn)*Mg/cations + nZn*Zn/cations]
    ZnMg[2*nZn+at] = 'O', A + [(at-nZn -wz2)*Mg/cations + nZn*Zn/cations]
    
ZnMg[-1] = 'O', A + [(-1 -wz2)*Mg/cations]
ZnMg[nMg+nZn] = 'O', C + [(-wz2)*Mg/cations]
ZnMg[nZn*2+nMg] = 'O', B + [(nZn - wz2)*Zn/cations]



ZnMg.to(fmt='poscar',filename='/home/nadamski/strain.vasp')